In [1]:
import requests
import time
import numpy as np
import pandas as pd
import copy

from keras.layers import Input, Embedding, Flatten, Dot, Concatenate, Dropout, Dense
from keras.models import Model

Using TensorFlow backend.


# Third environmement

## Get The Data

In [2]:
#attributes fa
USER_ID = 'IAHIZPIW80WPGDW7P7JE'
BASE_URL = "http://35.180.178.243"
url_reset = BASE_URL + "/reset"
url_predict = BASE_URL + "/predict"
params = {"user_id" : USER_ID}

def load_data(url_reset,params) :
    r = requests.get(url=url_reset, params=params)
    data = r.json()
    return data

data = load_data(url_reset,params)
nb_users = data["nb_users"]
nb_items = data["nb_items"]
next_state = data["next_state"]

data.keys()

dict_keys(['action_history', 'nb_items', 'nb_users', 'next_state', 'rewards_history', 'state_history'])

In [3]:
# # #See the request predict from our API
# params = {"user_id" : user_id,"recommended_item": 2}
# requests.get(url=url_predict, params=params).json()

In [4]:
blacklisted_set = set(("next_state","nb_items","nb_users"))
new_dict = {key : value for key, value in data.items() if key not in blacklisted_set}

df = pd.DataFrame(new_dict)
df.head()

,action_history,rewards_history,state_history
0,9,0.000000,"[[36, 0, 431.715260015307, 0.9136113202751137,..."
1,4,0.000000,"[[22, 0, 431.715260015307, 0.7639212155176396,..."
2,7,0.000000,"[[78, 0, 431.715260015307, 0.1533502480463882,..."
3,29,113.081583,"[[5, 0, 431.715260015307, 0.25319661091464896,..."
4,4,0.000000,"[[75, 0, 431.715260015307, 0.7772904200674785,..."


In [5]:
df.describe()

,action_history,rewards_history
count,200.000000,200.000000
mean,14.210000,108.771438
std,8.428589,243.844696
min,0.000000,0.000000
25%,6.750000,0.000000
50%,14.000000,0.000000
75%,21.000000,0.000000
max,29.000000,982.671823


In [6]:
df[df['rewards_history']==0].count()

action_history     156
rewards_history    156
state_history      156
dtype: int64

In [7]:
users_items_df = []
for i in range(df.shape[0]):
    users_items_df += df['state_history'][i] 
    
users_items_df = pd.DataFrame(users_items_df,columns = ['user_id','item_id',
                                                        'price','v1','v2','v3','v4','v5'])
users_items_df

,user_id,item_id,price,v1,v2,v3,v4,v5
0,36,0,431.715260,0.913611,0.245992,0.193816,2.756896,1.008183
1,36,1,657.000117,0.913611,0.245992,0.598705,0.499659,1.783631
2,36,2,380.046806,0.913611,0.245992,2.338361,0.349464,2.200802
3,36,3,220.205662,0.913611,0.245992,-0.310786,0.234045,-0.618043
4,36,4,31.062136,0.913611,0.245992,0.123133,1.436098,1.186815
5,36,5,461.783725,0.913611,0.245992,1.587258,1.230895,-0.269951
6,36,6,978.504194,0.913611,0.245992,-0.572030,0.205803,1.239934
7,36,7,494.500368,0.913611,0.245992,2.894246,0.740143,0.199190
8,36,8,392.776779,0.913611,0.245992,0.262130,-0.645260,2.897554
9,36,9,982.671823,0.913611,0.245992,-1.125255,1.991306,0.541967


In [8]:
# We can see that the first two covariates variables are features about users 
# and the forth and fifth variable are features about items
# The variable 5 depends on .....

user_v1 = users_items_df['v1'].unique()
user_v2 = users_items_df['v2'].unique()
users_ids = users_items_df['user_id'].unique()
users_content= pd.DataFrame({'user_id' : users_ids,'v1':user_v1,'v2':user_v2 })

item_v3 = users_items_df['v3'].unique()
item_v4 = users_items_df['v4'].unique()
item_id = users_items_df['item_id'].unique()
items_content= pd.DataFrame({'item_id' : item_id,'v3':item_v3,'v4':item_v4 })

## Data Preprocessing

In [9]:
pos_df = df[df["rewards_history"] > 0].reset_index(drop=True)
neg_df = df[df["rewards_history"] == 0].reset_index(drop=True)

In [10]:
pos_df["user_id"] = [pos_df["state_history"][i][0][0] for i in range(pos_df.shape[0])]
pos_df.head()## There are some users that have more than one positive item

,action_history,rewards_history,state_history,user_id
0,29,113.081583,"[[5, 0, 431.715260015307, 0.25319661091464896,...",5
1,9,982.671823,"[[81, 0, 431.715260015307, 1.6612831675612696,...",81
2,5,461.783725,"[[37, 0, 431.715260015307, 2.178404367619431, ...",37
3,24,43.080737,"[[5, 0, 431.715260015307, 0.25319661091464896,...",5
4,24,43.080737,"[[8, 0, 431.715260015307, 1.55670750676844, 1....",8


In [11]:
neg_df["user_id"] = [neg_df["state_history"][i][0][0] for i in range(neg_df.shape[0])]
neg_df.head()

,action_history,rewards_history,state_history,user_id
0,9,0.0,"[[36, 0, 431.715260015307, 0.9136113202751137,...",36
1,4,0.0,"[[22, 0, 431.715260015307, 0.7639212155176396,...",22
2,7,0.0,"[[78, 0, 431.715260015307, 0.1533502480463882,...",78
3,4,0.0,"[[75, 0, 431.715260015307, 0.7772904200674785,...",75
4,3,0.0,"[[34, 0, 431.715260015307, 0.19634282824106852...",34


# Model

In [12]:
import tensorflow as tf

def identity_loss(y_true, y_pred):
    """Ignore y_true and return the mean of y_pred
    
    This is a hack to work-around the design of the Keras API that is
    not really suited to train networks with a triplet loss by default.
    """
    return tf.reduce_mean(y_pred + 0 * y_true)


def margin_comparator_loss(inputs, margin=1.):
    """Comparator loss for a pair of precomputed similarities
    
    If the inputs are cosine similarities, they each have range in
    (-1, 1), therefore their difference have range in (-2, 2). Using
    a margin of 1. can therefore make sense.

    If the input similarities are not normalized, it can be beneficial
    to use larger values for the margin of the comparator loss.
    """
    positive_pair_sim, negative_pair_sim = inputs
    return tf.maximum(negative_pair_sim - positive_pair_sim + margin, 0)

## Naive Model

In [16]:
def naive_model(state):
    greater_index = 0
    greater_value = 0
    
    if i in range(len(state)):
        if(state[i][2] > greater_value):
            greater_value = state[i][2]
            greater_index = i
    return greater_index  

In [21]:
number_run = 3
n_epochs = 1000
total_reward = 0

for k in range(number_run):
    
    for i in range(n_epochs):

        predicted_item = naive_model(next_state)

        params['recommended_item'] = predicted_item

        r = requests.get(url = url_predict, params=params).json()
        reward = r['reward']
        total_reward += reward
        next_state = r['state']
        
        
print("Total reward: %f and Total reward per run: %f"%(total_reward, total_reward/number_run,))

Total reward: 305808.927461 and Total reward per run: 101936.309154


## Model 1

In [13]:
from keras.models import Model
from keras.layers import Embedding, Flatten, Input, Dense
from keras.layers import Lambda, Dot
from keras.regularizers import l2
from keras.layers.merge import dot, concatenate


def build_models(n_users, n_items, latent_dim=64, l2_reg=0):
    """Build a triplet model and its companion similarity model
    
    The triplet model is used to train the weights of the companion
    similarity model. The triplet model takes 1 user, 1 positive item
    (relative to the selected user) and one negative item and is
    trained with comparator loss.
    
    The similarity model takes one user and one item as input and return
    compatibility score (aka the match score).
    """
    # Common architectural components for the two models:
    # - symbolic input placeholders
    user_input = Input((1,), name='user_input')
    positive_item_input = Input((1,), name='positive_item_input')
    negative_item_input = Input((1,), name='negative_item_input')

    # - embeddings
    l2_reg = None if l2_reg == 0 else l2(l2_reg)
    user_layer = Embedding(input_dim=n_users, output_dim=latent_dim, input_length=1,
                           name='user_embedding', embeddings_regularizer=l2_reg)
    
    # The following embedding parameters will be shared to encode both
    # the positive and negative items.
    item_layer = Embedding(input_dim=n_items, output_dim=latent_dim, input_length=1,
                           name="item_embedding", embeddings_regularizer=l2_reg)

    user_embedding = Flatten()(user_layer(user_input))
    positive_item_embedding = Flatten()(item_layer(positive_item_input))
    negative_item_embedding = Flatten()(item_layer(negative_item_input))

    # - similarity computation between embeddings
    positive_similarity = Dot(name="positive_similarity",
                              axes=1, normalize=True)(
        [user_embedding, positive_item_embedding])
    negative_similarity = Dot(name="negative_similarity",
                              axes=1, normalize=True)(
        [user_embedding, negative_item_embedding])

    # The triplet network model, only used for training
    triplet_loss = Lambda(margin_comparator_loss,
                          name='comparator_loss',
                          output_shape=(1,))([positive_similarity, negative_similarity])

    triplet_model = Model(inputs=[user_input,
                                  positive_item_input,
                                  negative_item_input],
                          outputs=triplet_loss)
    
    # The match-score model, only use at inference to rank items for a given
    # model: the model weights are shared with the triplet_model therefore
    # we do not need to train it and therefore we do not need to plug a loss
    # and an optimizer.
    match_model = Model(inputs=[user_input, positive_item_input],
                        outputs=positive_similarity)
    
    return triplet_model, match_model


In [14]:
def sample_triplets(pos_df,max_items):
    """
    Sample negatives at random
    
    """
    range_itens = np.arange(max_items)
    user_ids = pos_df['user_id'].values
    pos_item_ids = pos_df['action_history'].values
    
    
    user_pos_items = pos_df.groupby('user_id')['action_history'].apply(list)
    neg_item_ids = np.array([])
    for i in user_ids:
        if i in set(user_pos_items.index):
            number = np.random.choice([item for item in range_itens 
                                       if not item in user_pos_items[user_pos_items.index == i]])
        else:
            number = np.random.choice(range_itens,1)
        neg_item_ids = np.append(neg_item_ids,number)
    
  
    return [user_ids, pos_item_ids, neg_item_ids]

In [ ]:
number_run = 3
n_epochs = 1
total_reward = 0
generations = 1000

for k in range(number_run):

    triplet_model, match_model = build_models(nb_users, nb_items, latent_dim=16, l2_reg=1e-6)
    # we plug the identity loss and the a fake target variable ignored by
    # the model to be able to use the Keras API to train the triplet model
    triplet_model.compile(loss=identity_loss, optimizer="adam")
    fake_y = np.ones_like(pos_df['user_id'])
    params = {'user_id': USER_ID}

    positive_data = pos_df.copy()

    for j in range(generations):

        for i in range(n_epochs):
            # Sample new negatives to build different triplets at each epoch
            triplet_inputs = sample_triplets(positive_data, max_items=nb_items)

            # Fit the model incrementally by doing a single pass over the
            # sampled triplets.
            triplet_model.fit(triplet_inputs, fake_y, shuffle=True, epochs=1)

        # The user that we need to recommend a product
        new_user_id = next_state[0][0]
        # Take all the items in the new state
        items_ids = np.array([next_state[i][1]
                              for i in range(len(next_state))])

        # Create an array of the same size of 'items_ids' of all values equal to 'new_user_id'
        repeated_user_id = np.empty_like(items_ids)
        repeated_user_id.fill(new_user_id)

        # predict the product using our model
        predicted = match_model.predict([repeated_user_id, items_ids])
        predicted_item = np.argmax(predicted)
        params['recommended_item'] = predicted_item

        # take the reward from the API
        r = requests.get(url=url_predict, params=params).json()
        reward = r['reward']
        total_reward += reward

        # Check if the reward is greater than 0 , if yes add to the 'positive_data'
        if(reward > 0):
            positive_data = positive_data.append({'action_history': predicted_item,
                                                  'state_history': next_state,
                                                  'user_id': next_state[0][0]},
                                                 ignore_index=True)

        fake_y = np.ones_like(positive_data['user_id'])
        # Take the nex_state
        next_state = r['state']


print("Total reward: %f and Total reward per run: %f" %(total_reward, total_reward/number_run,))

Epoch 1/1
44/44 [==============================] - 0s 9ms/step - loss: 0.9740
Epoch 1/1
45/45 [==============================] - 0s 99us/step - loss: 0.8877
Epoch 1/1
45/45 [==============================] - 0s 133us/step - loss: 0.8148
Epoch 1/1
45/45 [==============================] - 0s 83us/step - loss: 0.8302
Epoch 1/1
45/45 [==============================] - 0s 112us/step - loss: 0.7735
Epoch 1/1
46/46 [==============================] - 0s 87us/step - loss: 0.6963
Epoch 1/1
46/46 [==============================] - 0s 117us/step - loss: 0.7526
Epoch 1/1
47/47 [==============================] - 0s 93us/step - loss: 0.7351
Epoch 1/1
47/47 [==============================] - 0s 211us/step - loss: 0.6535
Epoch 1/1
47/47 [==============================] - 0s 91us/step - loss: 0.6119
Epoch 1/1
47/47 [==============================] - 0s 89us/step - loss: 0.6249
Epoch 1/1
47/47 [==============================] - 0s 131us/step - loss: 0.5495
Epoch 1/1
47/47 [==============================]

70/70 [==============================] - 0s 77us/step - loss: 0.1121
Epoch 1/1
70/70 [==============================] - 0s 87us/step - loss: 0.1528
Epoch 1/1
70/70 [==============================] - 0s 77us/step - loss: 0.1528
Epoch 1/1
70/70 [==============================] - 0s 91us/step - loss: 0.1284
Epoch 1/1
70/70 [==============================] - 0s 85us/step - loss: 0.1229
Epoch 1/1
70/70 [==============================] - 0s 85us/step - loss: 0.2065
Epoch 1/1
70/70 [==============================] - 0s 85us/step - loss: 0.1361
Epoch 1/1
70/70 [==============================] - 0s 81us/step - loss: 0.1217
Epoch 1/1
70/70 [==============================] - 0s 78us/step - loss: 0.1523
Epoch 1/1
70/70 [==============================] - 0s 81us/step - loss: 0.2182
Epoch 1/1
70/70 [==============================] - 0s 77us/step - loss: 0.1145
Epoch 1/1
70/70 [==============================] - 0s 85us/step - loss: 0.1615
Epoch 1/1
70/70 [==============================] - 0s 80us/ste

96/96 [==============================] - 0s 57us/step - loss: 0.1803
Epoch 1/1
96/96 [==============================] - 0s 59us/step - loss: 0.1382
Epoch 1/1
96/96 [==============================] - 0s 64us/step - loss: 0.2249
Epoch 1/1
96/96 [==============================] - 0s 56us/step - loss: 0.1982
Epoch 1/1
96/96 [==============================] - 0s 57us/step - loss: 0.1782
Epoch 1/1
97/97 [==============================] - 0s 73us/step - loss: 0.1407
Epoch 1/1
97/97 [==============================] - 0s 70us/step - loss: 0.2247
Epoch 1/1
97/97 [==============================] - 0s 70us/step - loss: 0.1079
Epoch 1/1
97/97 [==============================] - 0s 73us/step - loss: 0.2079
Epoch 1/1
97/97 [==============================] - 0s 71us/step - loss: 0.1686
Epoch 1/1
97/97 [==============================] - 0s 72us/step - loss: 0.1682
Epoch 1/1
98/98 [==============================] - 0s 69us/step - loss: 0.2162
Epoch 1/1
98/98 [==============================] - 0s 68us/ste

126/126 [==============================] - 0s 53us/step - loss: 0.2597
Epoch 1/1
126/126 [==============================] - 0s 60us/step - loss: 0.2269
Epoch 1/1
126/126 [==============================] - 0s 55us/step - loss: 0.1815
Epoch 1/1
126/126 [==============================] - 0s 54us/step - loss: 0.2512
Epoch 1/1
126/126 [==============================] - 0s 61us/step - loss: 0.2086
Epoch 1/1
126/126 [==============================] - 0s 55us/step - loss: 0.2092
Epoch 1/1
127/127 [==============================] - 0s 54us/step - loss: 0.2062
Epoch 1/1
127/127 [==============================] - 0s 54us/step - loss: 0.2562
Epoch 1/1
127/127 [==============================] - 0s 55us/step - loss: 0.2537
Epoch 1/1
127/127 [==============================] - 0s 58us/step - loss: 0.2167
Epoch 1/1
128/128 [==============================] - 0s 55us/step - loss: 0.2818
Epoch 1/1
128/128 [==============================] - 0s 55us/step - loss: 0.2042
Epoch 1/1
129/129 [===================

158/158 [==============================] - 0s 53us/step - loss: 0.1349
Epoch 1/1
159/159 [==============================] - 0s 58us/step - loss: 0.2388
Epoch 1/1
159/159 [==============================] - 0s 51us/step - loss: 0.2249
Epoch 1/1
160/160 [==============================] - 0s 53us/step - loss: 0.2345
Epoch 1/1
161/161 [==============================] - 0s 66us/step - loss: 0.2384
Epoch 1/1
161/161 [==============================] - 0s 59us/step - loss: 0.2436
Epoch 1/1
161/161 [==============================] - 0s 61us/step - loss: 0.3036
Epoch 1/1
161/161 [==============================] - 0s 63us/step - loss: 0.2258
Epoch 1/1
161/161 [==============================] - 0s 59us/step - loss: 0.2177
Epoch 1/1
161/161 [==============================] - 0s 59us/step - loss: 0.2139
Epoch 1/1
161/161 [==============================] - 0s 60us/step - loss: 0.1942
Epoch 1/1
161/161 [==============================] - 0s 63us/step - loss: 0.1876
Epoch 1/1
161/161 [===================

184/184 [==============================] - 0s 51us/step - loss: 0.2833
Epoch 1/1
185/185 [==============================] - 0s 51us/step - loss: 0.2545
Epoch 1/1
185/185 [==============================] - 0s 52us/step - loss: 0.2338
Epoch 1/1
186/186 [==============================] - 0s 55us/step - loss: 0.2800
Epoch 1/1
186/186 [==============================] - 0s 54us/step - loss: 0.2488
Epoch 1/1
187/187 [==============================] - 0s 51us/step - loss: 0.2498
Epoch 1/1
188/188 [==============================] - 0s 52us/step - loss: 0.2223
Epoch 1/1
188/188 [==============================] - 0s 51us/step - loss: 0.2521
Epoch 1/1
188/188 [==============================] - 0s 52us/step - loss: 0.2228
Epoch 1/1
188/188 [==============================] - 0s 51us/step - loss: 0.2498
Epoch 1/1
188/188 [==============================] - 0s 50us/step - loss: 0.2634
Epoch 1/1
188/188 [==============================] - 0s 49us/step - loss: 0.2943
Epoch 1/1
188/188 [===================

## Model 2

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Embedding, Flatten, Input, Dense, Dropout
from keras.layers import Concatenate, Lambda
from keras.regularizers import l2


def make_interaction_mlp(input_dim, n_hidden=1, hidden_size=64,
                         dropout=0, l2_reg=None):
    """
    Build the shared multi layer perceptron

    """
    mlp = Sequential()
    if n_hidden == 0:
        # Plug the output unit directly: this is a simple
        # linear regression model. Not dropout required.
        mlp.add(Dense(1, input_dim=input_dim,
                      activation='relu', kernel_regularizer=l2_reg))
    else:
        mlp.add(Dense(hidden_size, input_dim=input_dim,
                      activation='relu', kernel_regularizer=l2_reg))
        mlp.add(Dropout(dropout))
        for i in range(n_hidden - 1):
            mlp.add(Dense(hidden_size, activation='relu',
                          kernel_regularizer=l2_reg))
            mlp.add(Dropout(dropout))
        mlp.add(Dense(1, activation='relu', kernel_regularizer=l2_reg))
    return mlp


def build_models(n_users, n_items, user_dim=32, item_dim=64,
                 n_hidden=1, hidden_size=64, dropout=0, l2_reg=0):
    """
    Build models to train a deep triplet network

    """
    user_input = Input((1,), name='user_input')
    positive_item_input = Input((1,), name='positive_item_input')
    negative_item_input = Input((1,), name='negative_item_input')
    positive_meta_input = Input((2,), name='positive_meta_input')
    negative_meta_input = Input((2,), name='negative_meta_input')

    l2_reg = None if l2_reg == 0 else l2(l2_reg)
    user_layer = Embedding(n_users, user_dim, input_length=1,
                           name='user_embedding', embeddings_regularizer=l2_reg)

    # The following embedding parameters will be shared to encode both
    # the positive and negative items.
    item_layer = Embedding(n_items, item_dim, input_length=1,
                           name="item_embedding", embeddings_regularizer=l2_reg)

    user_embedding = Flatten()(user_layer(user_input))
    positive_item_embedding = Flatten()(item_layer(positive_item_input))
    negative_item_embedding = Flatten()(item_layer(negative_item_input))

    # Similarity computation between embeddings using a MLP similarity
    positive_embeddings_pair = Concatenate(name="positive_embeddings_pair")(
        [user_embedding, positive_item_embedding, positive_meta_input])
    positive_embeddings_pair = Dropout(dropout)(positive_embeddings_pair)

    negative_embeddings_pair = Concatenate(name="negative_embeddings_pair")(
        [user_embedding, negative_item_embedding, negative_meta_input])
    negative_embeddings_pair = Dropout(dropout)(negative_embeddings_pair)

    # Instanciate the shared similarity architecture
    interaction_layers = make_interaction_mlp(
        user_dim + item_dim + 2, n_hidden=n_hidden, hidden_size=hidden_size,
        dropout=dropout, l2_reg=l2_reg)

    positive_similarity = interaction_layers(positive_embeddings_pair)
    negative_similarity = interaction_layers(negative_embeddings_pair)

    # The triplet network model, only used for training
    triplet_loss = Lambda(margin_comparator_loss, output_shape=(1,),
                          name='comparator_loss')(
        [positive_similarity, negative_similarity])

    deep_triplet_model = Model(inputs=[user_input,
                                       positive_item_input,
                                       negative_item_input,
                                       positive_meta_input,
                                       negative_meta_input
                                       ],
                               outputs=[triplet_loss])

    # The match-score model, only used at inference
    deep_match_model = Model(inputs=[user_input, positive_item_input, positive_meta_input],
                             outputs=[positive_similarity])

    return deep_match_model, deep_triplet_model

In [ ]:
def sample_triplets(pos_df, items_content, max_items):
    """
    Sample the data and put in the format 
    [user_ids, pos_item_ids, neg_item_ids,pos_item_metadata,neg_item_metadata]

    """
    range_itens = np.arange(max_items)
    user_ids = pos_df['user_id'].values
    pos_item_ids = pos_df['action_history'].values

    pos_item_metadata = items_content.loc[pos_item_ids][['v3', 'v4']].values

    # TODO
    user_pos_items = pos_df.groupby('user_id')['action_history'].apply(list)
    neg_item_ids = np.array([])
    for i in user_ids:
        if i in set(user_pos_items.index):
            number = np.random.choice([item for item in range_itens 
                                       if not item in user_pos_items[user_pos_items.index == i]])
        else:
            number = np.random.choice(range_itens, 1)
        neg_item_ids = np.append(neg_item_ids, number)

    neg_item_metadata = items_content.loc[neg_item_ids][['v3', 'v4']].values

    return [user_ids, pos_item_ids, neg_item_ids, pos_item_metadata, neg_item_metadata]

In [ ]:
hyper_parameters = dict(
    user_dim=50,
    item_dim=16,
    n_hidden=1,
    hidden_size=16,
    dropout=0.2,
    l2_reg=1
)

In [ ]:
number_run = 3
n_epochs = 1
total_reward = 0
generations = 1000


for k in range(number_run):

    deep_match_model, deep_triplet_model = build_models(nb_users, nb_items,**hyper_parameters)
    # we plug the identity loss and the a fake target variable ignored by
    # the model to be able to use the Keras API to train the triplet model
    deep_triplet_model.compile(loss=identity_loss, optimizer="adam")
    fake_y = np.ones_like(pos_data['user_id'])
    params = {'user_id': USER_ID}
    
    positive_data = pos_df.copy()

    for j in range(generations):

        for i in range(n_epochs):

            # Sample new negatives to build different triplets at each epoch
            triplet_inputs = sample_triplets(
                positive_data, items_content, max_items=nb_items)

            # Fit the model incrementally by doing a single pass over the
            # sampled triplets.
            deep_triplet_model.fit(triplet_inputs, fake_y, shuffle=True,
                                   batch_size=64, epochs=1)

        # Predicting
        new_user_id = next_state[0][0]
        items_ids = np.array([next_state[i][1]
                              for i in range(len(next_state))])
        repeated_user_id = np.empty_like(items_ids)
        repeated_user_id.fill(new_user_id)

        pos_metadata = items_content.loc[items_ids][['v3', 'v4']].values
        predicted = deep_match_model.predict(
            [repeated_user_id, items_ids, pos_metadata])
        predicted_item = np.argmax(predicted)
        params['recommended_item'] = predicted_item

        r = requests.get(url=url_predict, params=params).json()
        reward = r['reward']
        total_reward += reward

        if(reward > 0):
            positive_data = positive_data.append({'action_history': predicted_item,
                                                  'state_history': next_state,
                                                  'user_id': next_state[0][0]},
                                                 ignore_index=True)

        fake_y = np.ones_like(positive_data['user_id'])
        next_state = r['state']

print("Total reward: %f and Total reward per run: %f"%(total_reward, total_reward/number_run,))